In [31]:
#Sawyer Thomas 
#University of Washington Transformative Robotics Lab
#code to determine the total valid combination count for an AxB linkage lattice
#improved algorithm to reduce computation time

import numpy as np
import numba
from numba import jit
import itertools

In [32]:
#numba functions with just in time computations for improved run time

@jit(nopython=True)
def set_ys(wide,ys,height):
    '''assign vertical distance value to each linkage
        inputs: wide: width of lattice (scalar), ys: empty 2d array, height: 1d array of heights
        outputs: filled 2d array'''
    for w in range(wide):
        np.append(ys,height)
    return ys

@jit(nopython=True)
def set_top_xs(xs,wide,charges,a,wid):
    '''set the horizontal distance value to each linkage joint for top linkages
        inputs: xs: 2d array of zeros with lattice dimensions, wide:width of lattice (scalar),
                charges: array of [-1,1..] charges, a: compressed hoizontal displacement, wid:linkage width
        outputs: partially filled 2d array of xs'''
    for p in range(1,wide):
        #add width of each additional compressed linkage
        xs[p][0]=xs[p-1][0]+wid
        if p%2==0:
            xs[p][0]+=(charges[p-1][0]*a-charges[p][0]*a)
    return xs

@jit(nopython=True)
def assign_arr_vals(xs,charges,tall,wide,a):
    ''' assign a horizontal distance value for each linkage joint in the lattice
        inputs: xs:2d array with partially filled x values, charges:array of [-1,1..] charges,
                tall: #vertical linkages in lattice (scalar),wide: # horizontal linkages in lattice, 
                a: compressed hoizontal displacement
        outputs: fully filled 2d array of xs'''
    for h in range(1,tall):
        for w in range(wide):
            #step down each column adding compressed vertical linkage displacement  
            xs[w,h]=xs[w,h-1]+charges[w,h-1]*a
    return xs

@jit(nopython=True)
def test_cond(xs,wide,tall,a,wid):
    '''check every other joint in the vertical direction to see if hoizontal link length
        are preserved throughout the entire structure
        inputs: xs: 2d array with filled in x values, wide: # horizontal linkages in lattice,
                tall: #vertical linkages in lattice (scalar),a: compressed hoizontal displacement
                wid:linkage width
        outputs: boolean 1 if valid, 0 if not valid'''
    for h in range(0,tall):
        for w in range(wide-1):
            if h%2==0 and w%2==0:
                if abs(xs[w,h]+wid-xs[w+1,h])>a/100:
                    return 0
            if h%2==1 and w%2==1:
                if abs(xs[w,h]+wid-xs[w+1,h])>a/100:
                    return 0
    else: return 1

In [33]:
# @jit(nopython=True)
def test_plot(charges,tall,wide):
    '''test whether a given lattice combination will have a valid trajectory
        inputs: a 1D array of [-1s and 1s], height and width of lattice
        outputs: boolean for lattice validity'''
    
    charges=np.array(charges)
    #cell dimensions
    a=.3 #horizontal displacment
    wid=1 #horizontal linkage length
    
    #set y values (these remain constant throughout)
    ys=np.array(())
    height=np.array(range(tall))
    ys=set_ys(wide,ys,height)
    
    #assign the top values
    xs=np.zeros((wide,tall))
    xs=set_top_xs(xs,wide,charges,a,wid)
    xs[0,0]=0
    
    #assign all other x vals
    xs=assign_arr_vals(xs,charges,tall,wide,a)
    
    #test to see if the array matches valid conditions
    test=test_cond(xs,wide,tall,a,wid)

    return test

In [34]:
def combos(hi):
    ''' use itertools to generate the cartesian product combinations for [-1,1] with a size of (height)
        inputs: # horizontal linkages (scalar)
        outputs: iterator that will return every combo when called'''
    
    x = [-1, 1]
    length = hi
    #dont turn into a list
    #just keep using as an iterator
    s=itertools.product(x, repeat=(hi))
    for shape in s:
        yield shape
    


    
def find_combos(hi,wid):
    ''' algorithm tests the validity of a lattice for every possible joint combination.
        Optimized version steps through columns to reduce total test count
        inputs: dimensions of lattice, height and width (scalars)
        outputs: total tested combination count, total valid combination count'''
    
    count=0
    success=0
    #generate all possible combos 
    next_combo=combos(hi)
    possible_sf=[] #list for possible column combos
    
    #generate all combinations for a single column 
    for m in range(2**(hi)):
        shape=list(next(next_combo))
        possible_sf.append([shape])

    #step through each column of the lattice    
    for n in range(wid-1):
        #print current layer for each new step
        print('layer is '+str(n+1))
        print('iteration count is currently = '+ str(count))
        
        possible_sf_new=[] #only record valid column options to test next
        #step through possible combos for a column
        for p in possible_sf:
            next_combo=combos(hi) #find all possible combos for the next column
            #iterate through all combos in the next column 
            for m in range(2**(hi)):
                if count%1000000==0:print(count) #print update every 1 million tests
                shape=list(next(next_combo)) #current shape to test
                count+=1 #increase total count
                iter_s=p.copy() #copy current shape
                iter_s.append(shape)
                
                #test the current shape
                s=test_plot(iter_s,hi+1,n+2)
                
                #if valid append to valid shape list
                if s ==1:
                    possible_sf_new.append(iter_s)
                    #only append valid states when considering the full lattice
                    if n ==wid-2:
                        success+=1
        #update the valid combos for the next column test
        possible_sf=possible_sf_new
        
    return count,success
    

'''enter the lattice size for an AxB linkage lattice'''    
A=4    
B=4

count,sucess=find_combos(A,B)
print('iteration count = '+ str(count))
print('success count = '+ str(success))


layer is 1
iteration count is currently = 0
0
layer is 2
iteration count is currently = 256
layer is 3
iteration count is currently = 832
iteration count = 4288
success count = 486
